In [1]:
import os, sys, json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from tqdm.notebook import tqdm


In [2]:
# Unix
# input
data_path = "/home/mayer/LocalRepository/JupyterProjects/DeepFashion2/dataset/image"
info_path = "/home/mayer/LocalRepository/JupyterProjects/DeepFashion2/dataset/annos"
# output
saveto = "/home/mayer/LocalRepository/JupyterProjects/DeepFashion2/dataset/masks"
segmentation_path = "/home/mayer/LocalRepository/JupyterProjects/DeepFashion2/dataset/segmentation.json"
#train_rec_file = os.path.join(data_path, 'train_recognition.json')
#train_rec_file_ext = os.path.join(data_path, 'train_recognitionExt.json')

#### Generate segmentation dataset

In [3]:
color_scheme = {1:(0,0,127),2:(0,0,255),
                3:(0,127,0),4:(0,127,127),5:(0,127,255),
                6:(0,255,0),7:(0,255,127),8:(0,255,255),
                9:(127,0,0),10:(127,0,127),11:(127,0,255),
                12:(127,127,0),13:(127,127,255)}
infosegmentation = []

for name in tqdm(os.listdir(data_path),desc="GENERATE MASKS"):
    id, ext = name.split(".")  
    with open(os.path.join(info_path, id+".json")) as rf:
        info = json.load(rf)
    image_path = os.path.join(data_path, name)
    #mask_path = os.path.join(data_path, id+"_mask."+ext)
    #image_savedebug = os.path.join(saveto, id+"_imgwithmask."+ext) 
    mask_path = os.path.join(saveto, id+"_mask."+ext)  
    infoimg = { "file": image_path, "mask": mask_path,"cloth":[]}    
    
    img = cv2.imread(image_path)
    if img is None:
        continue
    mask = np.zeros(shape=img.shape, dtype=np.uint8)    
    # for all clothes
    index=1
    item = "item"+str(index)
    
    # debug
    #print("\n\n\n\n\n\n\n"+name)
    #for key in info:
    #    print("\n\n"+key+":")
    #    print(info[key])
    ###
    
    while(info.get(item)!=None):
        # for all part of cloth
        infoitem = {}
        category = info[item]["category_id"]
        infoitem["category_id"] = category
        infoitem["areas"]=[]
        for area in info[item]["segmentation"]:
            # теряю точность тк cv2 не хочет работать с флотами
            area = np.array(list(map(int,area))).reshape(-1,2)
            #print(area)
            #cv2.fillConvexPoly(mask, area, color_scheme[category])     
            #test =  np.array([[100,100],[100,200],[200,200],[150,150],[190,160],[200,100],[100,100]])#квадраты
            #cv2.polylines(mask, np.int32([area]),True, color_scheme[category], 1)
            #cv2.polylines(mask, [area] ,True, color_scheme[category], 1)
            cv2.fillPoly(mask, [area], color_scheme[category])
            #cv2.fillPoly(img, [area], color_scheme[category])
            infoitem["areas"].append(area.tolist())         
            
        infoimg["cloth"].append(infoitem)
        index+=1
        item = "item"+str(index)
        
    cv2.imwrite(mask_path, mask)  
    #cv2.imwrite(image_savedebug, img)  
    infosegmentation.append(infoimg)

In [4]:
with open(segmentation_path, 'w') as wf:
    json.dump(infosegmentation, wf)